In [ ]:
import cv2
import os
import dlib
import numpy as np

# Paths to directories
known_faces_dir = r"path to directory where the registered faces will be stored"

# Ensure the known faces directory exists
if not os.path.exists(known_faces_dir):
    os.makedirs(known_faces_dir)

# Initialize dlib's face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor_path = r"path to shape_predictor_68_face_landmarks.dat"  # Ensure this path is correct

# Check if the predictor file exists
if not os.path.exists(predictor_path):
    raise FileNotFoundError(f"Predictor file not found at path: {predictor_path}")

predictor = dlib.shape_predictor(predictor_path)

def capture_known_faces(person_name):
    person_dir = os.path.join(known_faces_dir, person_name)
    if not os.path.exists(person_dir):
        os.makedirs(person_dir)

    cap = cv2.VideoCapture(0)

    print("Press 'q' to quit and save the images.")

    count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray)

        for face in faces:
            x, y, w, h = (face.left(), face.top(), face.width(), face.height())
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        cv2.imshow('Capture Faces', frame)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

        if len(faces) > 0:
            face = faces[0]
            x, y, w, h = (face.left(), face.top(), face.width(), face.height())
            face_image = frame[y:y + h, x:x + w]
            face_filename = os.path.join(person_dir, f"{person_name}_{count}.jpg")
            cv2.imwrite(face_filename, face_image)
            count += 1

    cap.release()
    cv2.destroyAllWindows()

# Capture faces for a new person
person_name = 'name'  # Change this to the name of the person you're capturing
capture_known_faces(person_name)


In [ ]:
import os
import cv2
import dlib
import numpy as np
import shutil


# Load pre-trained face detector and shape predictor
detector = dlib.get_frontal_face_detector()
predictor_path = r"path to shape_predictor_68_face_landmarks.dat" 
predictor = dlib.shape_predictor(predictor_path)

# Load the face recognition model
face_rec_model_path = r"path to dlib_face_recognition_resnet_model_v1.dat"
face_rec_model = dlib.face_recognition_model_v1(face_rec_model_path)

# Function to extract face descriptor
def get_face_descriptor(image, face):
    shape = predictor(image, face)
    return np.array(face_rec_model.compute_face_descriptor(image, shape))

# Paths to directories
known_faces_dir = r"path to the known faces directory"
new_album_dir = r"path to the album which has to be segregated"
output_base_dir = r'path to the folder where the segregated images will be saved'

# Ensure the output directory exists
if not os.path.exists(output_base_dir):
    os.makedirs(output_base_dir)

# Load known faces from detected_faces
known_face_descriptors = []
known_face_names = []

for person_name in os.listdir(known_faces_dir):
    person_dir = os.path.join(known_faces_dir, person_name)
    if os.path.isdir(person_dir):
        for filename in os.listdir(person_dir):
            filepath = os.path.join(person_dir, filename)
            if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                image = cv2.imread(filepath)
                faces = detector(image, 1)
                if len(faces) == 1:
                    descriptor = get_face_descriptor(image, faces[0])
                    known_face_descriptors.append(descriptor)
                    known_face_names.append(person_name)

# Function to detect and recognize faces in a new album
def detect_and_recognize_faces(image_path, threshold=0.6):
    image = cv2.imread(image_path)
    faces = detector(image, 1)  # Higher upsample for better detection
    
    face_labels = []
    face_confidences = []
    for face in faces:
        descriptor = get_face_descriptor(image, face)
        if len(known_face_descriptors) > 0:
            distances = np.linalg.norm(known_face_descriptors - descriptor, axis=1)
            min_distance_index = np.argmin(distances)
            confidence = 1 - distances[min_distance_index]
            if distances[min_distance_index] < threshold:  # Threshold for face match
                name = known_face_names[min_distance_index]
            else:
                name = "Unknown"
        else:
            name = "Unknown"
            confidence = 0.0
        
        face_labels.append(name)
        face_confidences.append(confidence)
    
    return face_labels, face_confidences

# Process the album
def process_album(input_dir, output_base_dir, album_name, threshold=0.6):
    output_dir = os.path.join(output_base_dir, album_name)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(input_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            image_path = os.path.join(input_dir, filename)
            face_labels, face_confidences = detect_and_recognize_faces(image_path, threshold)
            
            # Move the entire image to the corresponding directories based on recognized faces
            unique_labels = set(face_labels)
            for label in unique_labels:
                if label != "Unknown":
                    label_dir = os.path.join(output_dir, label)
                    if not os.path.exists(label_dir):
                        os.makedirs(label_dir)
                    
                    # Copy the entire image to the corresponding directory
                    destination_path = os.path.join(label_dir, filename)
                    if not os.path.exists(destination_path):  # Avoid copying the same file multiple times
                        shutil.copy(image_path, destination_path)

# Album name for organizing faces
album_name = 'album_matches'

# Process the album and organize detected faces
process_album(new_album_dir, output_base_dir, album_name)
